In [1]:
import data
import gensim
import neural_net
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
import torch.nn.functional as F

In [2]:
train_df, test_df = data.get_data()

In [3]:
train_sentences = train_df.sentence.values
train_labels = train_df.label.values

test_sentences = test_df.sentence.values
test_labels = test_df.label.values

num_words = 1000

In [4]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../model/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
def bring_nn_input(sentences):
    vec_list = []

    for sentence in sentences:
        words = sentence.strip().split()

        count = 0
        sum_vec = np.zeros(300)

        for word in words:
            if word in word2vec_model:
                sum_vec += word2vec_model[word]
                count += 1
        
        if count != 0:
            vec = (sum_vec / count)
    
        vec_list.append(vec)

    return np.array(vec_list)

In [6]:
train_vec = bring_nn_input(train_sentences)
test_vec = bring_nn_input(train_sentences)

In [17]:
# Building model
class Model(nn.Module):
    def __init__(self, input, hidden, output):
        super(Model, self).__init__()
        self.l1 = nn.Linear(input, hidden)
        self.l2 = nn.Linear(hidden , hidden)
        self.l3 = nn.Linear(hidden, output)

        # self.linear1 = torch.nn.Linear(input, hidden)
        # self.activation = torch.nn.ReLU()
        # self.linear2 = torch.nn.Linear(hidden, output)
        # self.softmax = torch.nn.Softmax()
    
    def forward(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = self.l3(out)

        return out

        # x = self.linear1(x)
        # x = self.activation(x)
        # x = self.linear2(x)
        # x = self.softmax(x)

        # return x

In [18]:
input = 300
hidden=100
output = 2

In [19]:
model = Model(input, hidden, output)

In [20]:
features_train, features_test, labels_train, labels_test = train_test_split(train_vec, train_labels, shuffle=True, random_state=34)

In [21]:
# params
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [22]:
def train(epochs):
    x_train = Variable(torch.from_numpy(features_train)).float()
    y_train = Variable(torch.from_numpy(labels_train)).long()
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        print ("epoch #",epoch)
        print ("loss: ", loss.item())
        pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
        print ("acc:(%) ", 100*pred/len(x_train))
        loss.backward()
        optimizer.step()

In [23]:
def test(epochs):
    model.eval()
    x_test = Variable(torch.from_numpy(features_test)).float()
    y_test = Variable(torch.from_numpy(labels_test)).long()
    for epoch in range(epochs):
        with torch.no_grad():
            y_pred = model(x_test)
            loss = criterion(y_pred, y_test)
            print ("epoch #",epoch)
            print ("loss: ", loss.item())
            pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
            print ("acc (%): ", 100*pred/len(x_test))

In [26]:
train(100)

/var/folders/x0/8r0rxn1n0zz_ffhdl9_ctgq40000gn/T/ipykernel_22555/3057272900.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


epoch # 0
loss:  0.4574134647846222
acc:(%)  tensor(85.8683)
epoch # 1
loss:  0.45703208446502686
acc:(%)  tensor(85.9688)
epoch # 2
loss:  0.4567093551158905
acc:(%)  tensor(85.9688)
epoch # 3
loss:  0.4563997983932495
acc:(%)  tensor(86.0510)
epoch # 4
loss:  0.45605918765068054
acc:(%)  tensor(86.0144)
epoch # 5
loss:  0.4557071328163147
acc:(%)  tensor(86.1332)
epoch # 6
loss:  0.45528513193130493
acc:(%)  tensor(86.0875)
epoch # 7
loss:  0.4548576772212982
acc:(%)  tensor(86.1971)
epoch # 8
loss:  0.4544161856174469
acc:(%)  tensor(86.2154)
epoch # 9
loss:  0.45400989055633545
acc:(%)  tensor(86.3159)
epoch # 10
loss:  0.45364469289779663
acc:(%)  tensor(86.3067)
epoch # 11
loss:  0.453305721282959
acc:(%)  tensor(86.3159)
epoch # 12
loss:  0.4530113935470581
acc:(%)  tensor(86.4712)
epoch # 13
loss:  0.4527553617954254
acc:(%)  tensor(86.3524)
epoch # 14
loss:  0.4525650441646576
acc:(%)  tensor(86.4438)
epoch # 15
loss:  0.45239296555519104
acc:(%)  tensor(86.4255)
epoch # 16
lo

In [27]:
test(100)

/var/folders/x0/8r0rxn1n0zz_ffhdl9_ctgq40000gn/T/ipykernel_22555/3057272900.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


epoch # 0
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 1
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 2
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 3
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 4
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 5
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 6
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 7
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 8
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 9
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 10
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 11
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 12
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 13
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 14
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
epoch # 15
loss:  0.4942275881767273
acc (%):  tensor(80.8715)
ep